In [2]:
from tensorflow.keras.layers import Input, Conv2D, Concatenate, MaxPool2D,\
     GlobalAvgPool2D, Activation
import tensorflow as tf

In [105]:
class Fire_Block(tf.keras.Model):
    def __init__(self, squeeze_filters, expand_filters):
        super(Fire_Block, self).__init__()
        self.a1  =  Conv2D(squeeze_filters, kernel_size=1,activation='relu')
        self.a2 = Conv2D(expand_filters, kernel_size=1, activation='relu')
        self.a3 = Conv2D(expand_filters, kernel_size=3, padding='same', activation='relu')
        self.concat = Concatenate()
    
    def call(self, inputs):
        x = self.a1(inputs)
        y = self.a2(x)
        z = self.a3(x)
        concat = self.concat([y,z])
        return concat

In [106]:
class SqueezeNet(tf.keras.Model):
    def __init__(self):
        super(SqueezeNet, self).__init__()
        self.conv = Conv2D(96,7,strides=2, padding='same', activation='relu')
        self.blocks = tf.keras.models.Sequential(name = 'SqueezeNet')
        sq_ch = 16
        ex_ch = 64
        max_list = [0,4,9]
        fire_list = [3,6,8]
        for i in range(11):
            if i in max_list:
                self.blocks.add(MaxPool2D(3, strides=2, padding='same'))
            else:
                if i in fire_list:
                    sq_ch = sq_ch  + 16
                    ex_ch = ex_ch  + 64
                self.blocks.add(Fire_Block(sq_ch, ex_ch))
    
        self.out = Conv2D(1000,1)
        self.globalpool = GlobalAvgPool2D()
        self.activation = Activation('softmax')

    def call(self,inputs):
        x = self.conv(inputs)
        x = self.blocks(x)
        x = self.out(x)
        x = self.globalpool(x)
        x = self.activation(x)

        return x

In [107]:
model = SqueezeNet()

In [108]:
model.build(input_shape=(None, 224, 224, 3))

In [109]:
model.summary()

Model: "squeeze_net_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_721 (Conv2D)          multiple                  14208     
_________________________________________________________________
SqueezeNet (Sequential)      multiple                  721216    
_________________________________________________________________
conv2d_746 (Conv2D)          multiple                  513000    
_________________________________________________________________
global_average_pooling2d_27  multiple                  0         
_________________________________________________________________
activation_27 (Activation)   multiple                  0         
Total params: 1,248,424
Trainable params: 1,248,424
Non-trainable params: 0
_________________________________________________________________
